In [ ]:
DATA_FOLDER = '/content/drive/MyDrive/PSTALN/data/'

In [ ]:
import pandas as pd

df_bibliovid = pd.read_json(DATA_FOLDER+'bibliovid.json')

In [ ]:
first_file_data = df_bibliovid.iloc[0]['abstract']
first_file_data

'Obesity is a risk factor for pneumonia and acute respiratory distress syndrome.\nTo determine whether obesity is associated with intubation or death, inflammation, cardiac injury, or fibrinolysis in coronavirus disease 2019 (COVID-19).\nRetrospective cohort study.\nA quaternary academic medical center and community hospital in New York City.\n2466 adults hospitalized with laboratory-confirmed severe acute respiratory syndrome coronavirus 2 infection over a 45-day period with at least 47 days of in-hospital observation.\nBody mass index (BMI), admission biomarkers of inflammation (C-reactive protein [CRP] level and erythrocyte sedimentation rate [ESR]), cardiac injury (troponin level), and fibrinolysis (D-dimer level). The primary end point was a composite of intubation or death in time-to-event analysis.\nOver a median hospital length of stay of 7 days (interquartile range, 3 to 14 days), 533 patients (22%) were intubated, 627 (25%) died, and 59 (2%) remained hospitalized. Compared wi

In [ ]:
df_bibliovid.iloc[0]['topics']

['Pronostique', 'Anesthésie-Réanimation', 'Infectiologie']

Erasing rows having no abstract

In [ ]:
df_bibliovid = df_bibliovid.dropna(subset=['abstract'])
df_bibliovid = df_bibliovid.drop(df_bibliovid[ df_bibliovid.abstract==''].index)

In [ ]:
df_bibliovid.head()

,id,slug,title,has_other_authors,impact_factor,goals_plain,verbose_date,authors,document_link,specialties,category,journal,link,results,synthesis,strength_of_evidence_details,goals,methods,pubmed_id,doi,abstract,topics,author_list,publication_date
0,769,body-mass-index-and-risk-for-intubation-or-dea...,Body Mass Index and Risk for Intubation or Dea...,True,"{'id': 3, 'name': 'Intermédiaire', 'posts_coun...",- Déterminer si l'obésité est associée à l'int...,31.07.2020,Anderson MR,https://www.acpjournals.org/doi/10.7326/M20-3214,"[{'id': 4, 'name': 'Anesthésie-Réanimation'}, ...","{'id': 6, 'name': 'Pronostique', 'icon': 'icon...","{'id': 41, 'name': 'Ann Intern Med'}",https://www.acpjournals.org/doi/10.7326/M20-3214,*Description de l'échantillon: 2112 patients c...,- Environ 2-3% des patients atteints de la COV...,-cohorte pronostique- puissance de l'étude sup...,- Déterminer si l'obésité est associée à l'int...,Cohorte rétrospective portant sur 2466 patient...,32726151,10.7326/M20-3214,Obesity is a risk factor for pneumonia and acu...,"[Pronostique, Anesthésie-Réanimation, Infectio...","[{'id': 714, 'name': 'Anderson MR'}]",2020-07-31
5,742,an-mrna-vaccine-against-sars-cov-2-preliminary...,An mRNA Vaccine against SARS-CoV-2 - Prelimina...,True,"{'id': 3, 'name': 'Intermédiaire', 'posts_coun...",Développement du vaccin accéléré mRNA-1273 Mod...,15.07.2020,Jackson LA,https://www.nejm.org/doi/10.1056/NEJMoa2022483,"[{'id': 22, 'name': 'Immunité'}, {'id': 5, 'na...","{'id': 4, 'name': 'Thérapeutique', 'icon': 'ic...","{'id': 22, 'name': 'NEJM'}",https://www.nejm.org/doi/10.1056/NEJMoa2022483,"Après la première vaccination, les réponses an...",Le vaccin mRNA-1273 est plutôt bien toléré. De...,Les résultats du rapport ne sont que prélimina...,Développement du vaccin accéléré mRNA-1273 Mod...,"Essai de vaccination de Phase 1, ouvert inclua...",32663912,10.1056/NEJMoa2022483,The severe acute respiratory syndrome coronavi...,"[Thérapeutique, Immunité, Virologie, Infectiol...","[{'id': 690, 'name': 'Jackson LA'}]",2020-07-15
6,739,pathophysiology-transmission-diagnosis-and-tre...,"Pathophysiology, Transmission, Diagnosis, and ...",True,"{'id': 2, 'name': 'Faible', 'posts_count': 505...",Etat des lieux bibliographique des connaissanc...,14.07.2020,Joost Wiersinga W,https://jamanetwork.com/journals/jama/fullarti...,"[{'id': 7, 'name': 'Transversale'}, {'id': 12,...","{'id': 2, 'name': 'Autres', 'icon': 'icon-other'}","{'id': 183, 'name': 'JAMA Network Open'}",https://jamanetwork.com/journals/jama/fullarti...,La transmission du SARS-CoV-2 est plus favorab...,Actualisation générale des connaissances (rech...,Revue orientée d'études pré-sélectionnées par ...,Etat des lieux bibliographique des connaissanc...,Bases de données indexées (générale et récente...,32648899,10.1001/jama.2020.12839,The coronavirus disease 2019 (COVID-19) pandem...,"[Autres, Transversale, Infectiologie]","[{'id': 687, 'name': 'Joost Wiersinga W'}]",2020-07-14
7,735,introductions-and-early-spread-of-sars-cov-2-i...,Introductions and Early Spread of SARS-CoV-2 i...,True,"{'id': 4, 'name': 'Indéterminé', 'posts_count'...",Etudier comment l'épidémie de SARS-Cov-2 a com...,10.07.2020,Gambaro F,https://www.eurosurveillance.org/content/10.28...,"[{'id': 7, 'name': 'Transversale'}, {'id': 5, ...","{'id': 5, 'name': 'Epidémiologique', 'icon': '...","{'id': 46, 'name': 'Eurosurveillance'}",https://www.eurosurveillance.org/content/10.28...,Le virus a été introduit plusieurs fois dans l...,Le virus SARS-Cov-2 a été introduit plusieurs ...,Les données de cette étude semblent disponible...,Etudier comment l'épidémie de SARS-Cov-2 a com...,Données. 97 séquences de SARS-Cov-2 recueillie...,32643599\n32289214\n32070465\n32109013\n321797...,10.2807/1560-7917.ES.2020.25.26.2001200,"Following SARS-CoV-2 emergence in China, a spe...","[Epidémiologique, Transversale, Virologie]","[{'id': 628, 'name': 'Gambaro F'}]",2020-07-10
12,724,how-to-safely-reopen-colleges-and-universities...,How to S

In [ ]:
def vectorize(label_list,dico):
    return [int(elt in label_list) for  elt in dico]

In [ ]:
spec_dict=dict()
spec_list=[]
for elt in df_bibliovid['specialties'].values:
    spec_list.append(elt[0]['name'])
spec_set=set(spec_list)
i=0
for elt in spec_set:
  spec_dict[elt]=i
  i+=1
print(spec_dict)

df_bibliovid['vect_specs'] = df_bibliovid.topics.apply(lambda x: vectorize(x,spec_dict))
df_bibliovid['cat_text'] = df_bibliovid['title'] + df_bibliovid['abstract']
df_bibliovid['len'] = df_bibliovid['cat_text'].apply(lambda x: len(x.split(' ')))
df_bibliovid = df_bibliovid.reset_index(drop=True)

{'Hépato-gastro-entérologie': 0, 'Immunité': 1, 'Anesthésie-Réanimation': 2, 'Psychiatrie': 3, 'Pédiatrie': 4, 'Hématologie': 5, 'Cardiologie et maladies métaboliques': 6, 'Neurologie': 7, 'Hygiène': 8, 'Néphrologie': 9, 'Confinement/Déconfinement': 10, 'Gynécologie Obstétrique': 11, 'Radiologie': 12, 'Gériatrie': 13, 'Transversale': 14, 'Infectiologie': 15, 'Virologie': 16}


Tokenizing for future uses with BERT

In [ ]:
!pip install -q transformers
import transformers
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from tqdm import tqdm

input_ids=[]
attention_mask=[]
token_type_ids=[]
def make_dict(l):
    d=dict()
    for elt in l:
      d[elt]=''
    return d
def remove_ponctuation(text,ponctuation):
    return ''.join([elt for elt in list(text) if elt not in ponctuation])
def remove_stopword(text):
    return ' '.join([word for word in nltk.word_tokenize((text.lower())) if word not in make_dict(stopwords.words('english'))])

ponctuation='''!()-[]{};:'"\,<>./?@#$%^&*_~'''
with  tqdm(total=df_bibliovid.shape[0], position=0) as progress:
          
  for index, row in df_bibliovid.iterrows(): #tqdm((df_litcovid.iterrows()),total=df_litcovid.shape[0],initial=0):
    x=tokenizer.encode_plus(
              remove_stopword(remove_ponctuation(row.cat_text,ponctuation)),
              add_special_tokens=True,
              max_length=200,
              pad_to_max_length=True,
              padding=False,
              return_token_type_ids=True
          )
    input_ids.append(x['input_ids'])
    attention_mask.append(x['attention_mask'])
    token_type_ids.append(x['token_type_ids'])
    progress.update()

  0%|          | 0/371 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 371/371 [00:13<00:00, 27.24it/s]


Only keeping data we're actually going to use

Adding columns to store some useful BERT data

In [ ]:
a = df_bibliovid.copy()
a['input_ids']=input_ids
a['attention_mask']=attention_mask
a['token_type_ids']=token_type_ids
a.head()

,id,slug,title,has_other_authors,impact_factor,goals_plain,verbose_date,authors,document_link,specialties,category,journal,link,results,synthesis,strength_of_evidence_details,goals,methods,pubmed_id,doi,abstract,topics,author_list,publication_date,vect_specs,cat_text,len,input_ids,attention_mask,token_type_ids
0,769,body-mass-index-and-risk-for-intubation-or-dea...,Body Mass Index and Risk for Intubation or Dea...,True,"{'id': 3, 'name': 'Intermédiaire', 'posts_coun...",- Déterminer si l'obésité est associée à l'int...,31.07.2020,Anderson MR,https://www.acpjournals.org/doi/10.7326/M20-3214,"[{'id': 4, 'name': 'Anesthésie-Réanimation'}, ...","{'id': 6, 'name': 'Pronostique', 'icon': 'icon...","{'id': 41, 'name': 'Ann Intern Med'}",https://www.acpjournals.org/doi/10.7326/M20-3214,*Description de l'échantillon: 2112 patients c...,- Environ 2-3% des patients atteints de la COV...,-cohorte pronostique- puissance de l'étude sup...,- Déterminer si l'obésité est associée à l'int...,Cohorte rétrospective portant sur 2466 patient...,32726151,10.7326/M20-3214,Obesity is a risk factor for pneumonia and acu...,"[Pronostique, Anesthésie-Réanimation, Infectio...","[{'id': 714, 'name': 'Anderson MR'}]",2020-07-31,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Body Mass Index and Risk for Intubation or Dea...,259,"[101, 2303, 3742, 5950, 3891, 20014, 19761, 35...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,742,an-mrna-vaccine-against-sars-cov-2-preliminary...,An mRNA Vaccine against SARS-CoV-2 - Prelimina...,True,"{'id': 3, 'name': 'Intermédiaire', 'posts_coun...",Développement du vaccin accéléré mRNA-1273 Mod...,15.07.2020,Jackson LA,https://www.nejm.org/doi/10.1056/NEJMoa2022483,"[{'id': 22, 'name': 'Immunité'}, {'id': 5, 'na...","{'id': 4, 'name': 'Thérapeutique', 'icon': 'ic...","{'id': 22, 'name': 'NEJM'}",https://www.nejm.org/doi/10.1056/NEJMoa2022483,"Après la première vaccination, les réponses an...",Le vaccin mRNA-1273 est plutôt bien toléré. De...,Les résultats du rapport ne sont que prélimina...,Développement du vaccin accéléré mRNA-1273 Mod...,"Essai de vaccination de Phase 1, ouvert inclua...",32663912,10.1056/NEJMoa2022483,The severe acute respiratory syndrome coronavi...,"[Thérapeutique, Immunité, Virologie, Infectiol...","[{'id': 690, 'name': 'Jackson LA'}]",2020-07-15,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",An mRNA Vaccine against SARS-CoV-2 - Prelimina...,269,"[101, 28848, 17404, 18906, 9363, 2615, 2475, 8...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,739,pathophysiology-transmission-diagnosis-and-tre...,"Pathophysiology, Transmission, Diagnosis, and ...",True,"{'id': 2, 'name': 'Faible', 'posts_count': 505...",Etat des lieux bibliographique des connaissanc...,14.07.2020,Joost Wiersinga W,https://jamanetwork.com/journals/jama/fullarti...,"[{'id': 7, 'name': 'Transversale'}, {'id': 12,...","{'id': 2, 'name': 'Autres', 'icon': 'icon-other'}","{'id': 183, 'name': 'JAMA Network Open'}",https://jamanetwork.com/journals/jama/fullarti...,La transmission du SARS-CoV-2 est plus favorab...,Actualisation générale des connaissances (rech...,Revue orientée d'études pré-sélectionnées par ...,Etat des lieux bibliographique des connaissanc...,Bases de données indexées (générale et récente...,32648899,10.1001/jama.2020.12839,The coronavirus disease 2019 (COVID-19) pandem...,"[Autres, Transversale, Infectiologie]","[{'id': 687, 'name': 'Joost Wiersinga W'}]",2020-07-14,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","Pathophysiology, Transmission, Diagnosis, and ...",436,"[101, 4130, 7361, 10536, 20763, 6483, 6726, 11...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,735,introductions-and-early-spread-of-sars-cov-2-i...,Introductions and Early Spread of SARS-CoV-2 i...,True,"{'id': 4, 'name': 'Indéterminé', 'posts_count'...",Etudier commen

Saving the preproccessed data

In [ ]:
import pickle

with open(DATA_FOLDER+'clean_df_bibliovid_pretreated.pkl', 'wb') as f:
  pickle.dump(a, f)